# GPT-4o image-to-text extraction

### Lab Notebook Preprocessing Pipeline:

1. Combine all lab member notebooks into one PDF file per notebook. 
2. Process each of the notebooks through the LLM extraction script (also available as command line).
3. (Future) Run the local Streamlit app **"Lab Note Finder"** in order to query the chatbot about lab notebooks. 

In [1]:
from pdf_to_text_gpt4o import convert_notebook_to_text

In [ ]:
notebook_pdf = "test/sample_notebook.pdf"
output_file = "test/sample_notebook.md"
book_name = "LF Book 1"
your_api_key = "sk-proj-0X07CUMUxTd3J6w......."

extraction_instructions = """
    You are a microbiology researcher who is tasked with transcribing all the handwritten notes 
    from images of lab notebook pages. Extract all handwritten text from this lab notebook page 
    and return it as markdown text. 
    Bold any text that looks like a date. 
    Italicise any text in Latin. 
    If a date looks like the start of a new notebook entry, then make it a header. 
    If you detect there is an agarose gel image, write a hash tag "#gel_image" for each gel image detected. 
    if you detect a hand-drawn agar plate diagram, write a hash tag "#plate_diagram" for each plate diagram detected. 
    There is a common strain named BJH001, sometimes just referred to as 001, which is referenced often.
    Lab members include Lance, Angela, Paul, Josie.
    Other common terms include Qiagen. 
    If you see a page number at the very bottom of the page, append it to every date header.
    """

In [7]:
# Extract markdown-formatted text from PDF file
notebook_text = convert_notebook_to_text(notebook_pdf, your_api_key, extraction_instructions, book_name)

# Save markdown text to file
with open(output_file, "w") as f: f.write(notebook_text)
print(f"Text extraction complete. Output saved to {output_file}")

Text extraction complete. Output saved to test/sample_notebook.md


In [8]:
print(notebook_text)

# **5/25/18** - Page 44
*Written in "LF Book 1":*

To figure out what is happening with my tests, I am doing protein precipitation from my evolved 001 per pH 4.6 as it is a well known toxin producer. I cultured 6 ml overnight and will run 5 precipitations:

1. Angela's sat. saturated *ammonium sulfate*  
   - I knew it worked before.
2. My *ammonium sulfate*  
   - It should be about same as Angela's.
3. 80% solid *ammonium sulfate*
4. 75% solid *ammonium sulfate*
5. 70% solid *ammonium sulfate*

Additionally, I am testing three different samples of BJH001 to find and use the one that kills on the lawns. It should unlike those used in the past several tests:

1. Ancestral 001
2. Smooth white
3. Yellow lumpy

#plate_diagram

# **5/26/18**
*Written in "LF Book 1":*

Results:  
Angela's supersaturated seems to be the best followed closely by my supersaturated and the 80% solid addition. 75% and 70% were highly smaller.

#plate_diagram

# **5/28/18**
*Written in "LF Book 1":*

Results:

Th

## Make vector database

In [9]:
from database import save_vector_store

save_vector_store(markdown_dir="./test", output_dir="./test/notebook_database")

In [10]:
from lab_rag_query import chat

chat(database_dir="./test/notebook_database", print_user_input=True)


What would you like to know about your lab's notebooks? (Type 'exit' to quit.)

================== You ==================

 what date, page, and notebook were the Qiagen dsRNA extractions performed?

================ Chatbot ================

The Qiagen dsRNA extraction was performed on # **6/14** (Page 55).

================== You ==================

 what notebook was that written in?

================ Chatbot ================

The Qiagen dsRNA extraction was written in the "LF Book 1" notebook.


### TODO in the backend:

1. Lab member opens Lab Note Finder app.
2. If a new lab notebook filename or file size change is detected in master data directory, trigger `update_database.py`.
    - Display *"Hang tight. Database is updating..."* message. 
    - Read each page of lab notebook by converting PDF page to image file. 
    - Stream image file in bytes to GPT-4o for markdown text conversion.
    - Write concatenated markdown text (whole notebook) to file.
    - Once all notebooks are updated, convert to FAISS database for efficient LLM searching.
    - When updates are complete, display *"Updates complete! Query away."*
4. User asks queries to chatbot on Streamlit interface who has memory of that session's conversation and can access the lab notebook FAISS database as a RAG application.
    - Make sure user query output includes quotes from lab notebook, lab notebook filename, and page number. 
    - There seems to be a problem with chunking. Figure out how to chunk better so the LLM has all relevant info. 
    - Ideally, each chunk should include the notebook owner, page, notebook ID, and date (including the year). 